In [3]:
# import requests
import requests
import json

In [4]:
# geo.admin lookup
# needs exact address

def get_lat_lon_wgs84(address):
    url = f"https://api3.geo.admin.ch/rest/services/api/SearchServer?searchText={address}&type=locations&origins=address"
    response = requests.get(url)
    data = response.json()

    return data["results"][0]["attrs"]["lat"], data["results"][0]["attrs"]["lon"]

get_lat_lon_wgs84("St. Leonhard-Strasse 45, St. Gallen")

(47.42184829711914, 9.369429588317871)

In [5]:
# get lat lon openstreetmap
def get_lat_lon(address):
    url = f"https://nominatim.openstreetmap.org/search?q={address}&format=json"
    response = requests.get(url)
    data = response.json()

    print(data)

    return data[0]["lat"], data[0]["lon"]

get_lat_lon("St. Leonhardstrasse 45, St. Gallen")

[{'place_id': 95207181, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 9227670034, 'boundingbox': ['47.4217008', '47.4218008', '9.369398', '9.369498'], 'lat': '47.4217508', 'lon': '9.369448', 'display_name': 'Leo&Co Coworking St. Gallen, 45, St. Leonhard-Strasse, Rosenberg, St. Gallen, Wahlkreis St. Gallen, Sankt Gallen, 9000, Schweiz/Suisse/Svizzera/Svizra', 'class': 'office', 'type': 'coworking', 'importance': 0.41009999999999996}, {'place_id': 44231395, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 3653216863, 'boundingbox': ['47.4217836', '47.4218836', '9.369338', '9.369438'], 'lat': '47.4218336', 'lon': '9.369388', 'display_name': '45, St. Leonhard-Strasse, Rosenberg, St. Gallen, Wahlkreis St. Gallen, Sankt Gallen, 9000, Schweiz/Suisse/Svizzera/Svizra', 'class': 'place', 'type': 'house', 'importance': 0.41009999999999996}]


('47.4217508', '9.369448')

In [6]:
def get_easting_northing_lv95(address):

    url = f"https://api3.geo.admin.ch//rest/services/api/SearchServer?lang=de&searchText={address}&type=locations&sr=2056"
    response = requests.get(url)
    data = response.json()

    return data["results"][0]["attrs"]["y"], data["results"][0]["attrs"]["x"]

get_easting_northing_lv95("St. Leonhard-Strasse 45, St. Gallen")

(2745681.0, 1254130.25)

In [7]:
# function to get heating info from long and lat

def get_heating_info(lat, lon, dist):
    url = f"https://daten.stadt.sg.ch/api/records/1.0/search/?dataset=warmeversorgung&q=&facet=waermevers&geofilter.distance={lat}%2C{lon}%2C{dist}"
    response = requests.get(url)
    data = response.json()
    return data

# function to get heating info from address

def get_heating_info_from_address(address, dist):
    lat, lon = get_lat_lon_wgs84(address)
    return get_heating_info(lat, lon, dist)

#get_heating_info_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)
get_heating_info_from_address("Möslenstrasse 3, St. Gallen", 1)

{'nhits': 1,
 'parameters': {'dataset': 'warmeversorgung',
  'rows': 10,
  'start': 0,
  'facet': ['waermevers'],
  'format': 'json',
  'geofilter.distance': ['47.41964340209961,9.382232666015625,1'],
  'timezone': 'UTC'},
 'records': [{'datasetid': 'warmeversorgung',
   'recordid': 'd2e9a2f8020d8d6698800869736ac3bb31f93e11',
   'fields': {'geo_shape': {'coordinates': [[[9.38289855470966,
        47.419466361952956],
       [9.382941120936318, 47.419410162001824],
       [9.38299121979418, 47.41942554941748],
       [9.383048303135295, 47.41944307750419],
       [9.383271772575283, 47.41951139107871],
       [9.383302981328848, 47.41952095138226],
       [9.383952215418173, 47.41970498620696],
       [9.384154446548276, 47.41974415485945],
       [9.384186992773554, 47.419755482790265],
       [9.384220370310745, 47.419765618124075],
       [9.384254511604873, 47.419774526010045],
       [9.38427317926302, 47.41977897105416],
       [9.38429199406562, 47.41978308972999],
       [9.3843

In [6]:
# function to get waermevers from address

def get_waermevers_from_address(address, dist):
    data = get_heating_info_from_address(address, dist)
    return data["records"][0]["fields"]["waermevers"]

get_waermevers_from_address("St. Leonhard-Strasse 45, St. Gallen", 1)

'Fernwärme Ergänzungsgebiet'

# data from geo.admin.ch

ch.bfe.solarenergie-eignung-daecher
ch.bfe.solarenergie-eignung-fassaden

# get data from geo.admin.ch

https://map.geo.admin.ch/?layers=ch.bfe.solarenergie-eignung-daecher&lang=en&topic=ech&bgLayer=ch.swisstopo.pixelkarte-farbe&E=2745681.00&N=1254117.67&zoom=11.371666666666666


In [25]:
# https://github.com/ThesisRocker/Data2DollarFS22/blob/0f7344650e6c6c43bc4057502493ea4afbce64f3/06_Abgabe%20Gruppenarbeit/Gruppe%201/sonnendach/script_solar_2.py

def get_geo_admin_data(address):
    addr_easting, addr_northing = get_easting_northing_lv95(address)

    # Uses the swiss coordinate system LV95
    url = f"https://api3.geo.admin.ch/rest/services/all/MapServer/identify?geometry={addr_easting},{addr_northing}&geometryFormat=geojson&geometryType=esriGeometryPoint&imageDisplay=1680,388,96&lang=en&layers=all:ch.bfe.solarenergie-eignung-daecher&limit=10&mapExtent=2745356.382965297,1254042.6972188372,2746005.609812927,1254192.6377050756&returnGeometry=true&sr=2056&tolerance=10"

    # Make the request to the API using requests library
    result_building = requests.get(url)

    # Transform result into json format
    result_building_json = result_building.json()

    return result_building_json["results"]


get_geo_admin_data("St. Leonhard-Strasse 45, St. Gallen")

[{'type': 'Feature',
  'featureId': 16745794,
  'bbox': [2745660.3, 1254105.7, 2745705.5, 1254142.5],
  'layerBodId': 'ch.bfe.solarenergie-eignung-daecher',
  'layerName': 'Solar energy: suitability of roofs',
  'id': 16745794,
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[2745702.8, 1254126.4],
      [2745671.5, 1254107.1],
      [2745669.2, 1254105.7],
      [2745660.3, 1254120.1],
      [2745696.6, 1254142.5],
      [2745705.5, 1254128.1],
      [2745702.8, 1254126.4]],
     [[2745661.8, 1254119.4],
      [2745669.5, 1254106.9],
      [2745704.1, 1254128.3],
      [2745696.4, 1254140.8],
      [2745661.8, 1254119.4]]]]},
  'properties': {'monate': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 12, 11],
   'building_id': 527069,
   'a_param': [0.7768488835,
    0.9652010208,
    0.9679495756,
    0.980280681,
    0.9901538902,
    0.9768872537,
    0.9585564666,
    0.9063184743,
    0.7307145044,
    0.3759406691,
    0.3489994673,
    0.5538456654],
   'b_param': [0.8689646264,
   

In [27]:
def get_links_to_sonnendach(data):
    links = []

    for result in data:
        id = result["id"]
        links.append(f"https://www.uvek-gis.admin.ch/BFE/sonnendach/?featureId={id}&lang=de")

    return links

get_links_to_sonnendach(get_geo_admin_data("St. Leonhard-Strasse 45, St. Gallen"))

['https://www.uvek-gis.admin.ch/BFE/sonnendach/?featureId=16745794&lang=de',
 'https://www.uvek-gis.admin.ch/BFE/sonnendach/?featureId=16745798&lang=de']

https://map.geo.admin.ch/?lang=de&topic=energie&bgLayer=ch.swisstopo.swissimage&catalogNodes=2419,2420,2427,2480,2429,2431,2434,2436,2767,2441,3206&layers=ch.swisstopo.amtliches-strassenverzeichnis,ch.bfe.solarenergie-eignung-daecher&zoom=12&X=254130&Y=745681&layers_opacity=0.85,0.65




In [37]:
def get_iframe_url(address):
    addr_easting, addr_northing = get_easting_northing_lv95(address)

    print(addr_easting, addr_northing)

    #url = f"<iframe src='https://map.geo.admin.ch/embed.html?lang=de&topic=energie&bgLayer=ch.swisstopo.swissimage&catalogNodes=2419,2420,2427,2480,2429,2431,2434,2436,2767,2441,3206&layers=ch.swisstopo.amtliches-strassenverzeichnis,ch.bfe.solarenergie-eignung-daecher&zoom=12&X=254130&Y=745681&layers_opacity=0.85,0.65' width='400' height='300' frameborder='0' style='border:0' allow='geolocation'></iframe>"
    url = f"https://map.geo.admin.ch/embed.html?lang=de&topic=energie&bgLayer=ch.swisstopo.swissimage&catalogNodes=2419,2420,2427,2480,2429,2431,2434,2436,2767,2441,3206&layers=ch.swisstopo.amtliches-strassenverzeichnis,ch.bfe.solarenergie-eignung-daecher&zoom=12&X={addr_northing}&Y={addr_easting}&layers_opacity=0.85,0.65"
    return url
    print(url)
    frame = f"<iframe src='{url}' width='400' height='300' frameborder='0' style='border:0' allow='geolocation'></iframe>"

get_iframe("St. Leonhard-Strasse 45, St. Gallen")

2745681.0 1254130.25
https://map.geo.admin.ch/embed.html?lang=de&topic=energie&bgLayer=ch.swisstopo.swissimage&catalogNodes=2419,2420,2427,2480,2429,2431,2434,2436,2767,2441,3206&layers=ch.swisstopo.amtliches-strassenverzeichnis,ch.bfe.solarenergie-eignung-daecher&zoom=12&X=254130&Y=745681&layers_opacity=0.85,0.65


In [49]:
from IPython.display import IFrame
IFrame(get_iframe_url("St. Leonhard-Strasse 45, St. Gallen"), width=600, height=450)

2745681.0 1254130.25


https://www.sg.ch/umwelt-natur/energie/Energiegesetz.html

